In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [4]:
data = pd.read_csv('data/data_fusion_little.csv')
data = data.dropna()
data.head()

,Titre,Description,Auteurs,Image,Lien Google,Editeur,Date publication,infoLink,Genre,Nb scores,Id,Prix,User_id,Nom lecteur,revue/utilité,revue/score,revue/heure,revue/résumé,revue/texte
8,The Church of Christ: A Biblical Ecclesiology ...,In The Church of Christ: A Biblical Ecclesiolo...,['Everett Ferguson'],http://books.google.com/books/content?id=kVqRa...,http://books.google.nl/books?id=kVqRaiPlx88C&p...,Wm. B. Eerdmans Publishing,1996,http://books.google.nl/books?id=kVqRaiPlx88C&d...,['Religion'],5.0,0802841899,25.97,ARI272XF8TOL4,Christopher J. Bray,74/81,5.0,955411200,Ecclesiological Milestone,With the publication of Everett Ferguson's boo...
9,The Church of Christ: A Biblical Ecclesiology ...,In The Church of Christ: A Biblical Ecclesiolo...,['Everett Ferguson'],http://books.google.com/books/content?id=kVqRa...,http://books.google.nl/books?id=kVqRaiPlx88C&p...,Wm. B. Eerdmans Publishing,1996,http://books.google.nl/books?id=kVqRaiPlx88C&d...,['Religion'],5.0,0802841899,25.97,A36TPZSH8LBT1,haskell,2/3,5.0,1311465600,Early Christian development of the Church,Everett Ferguson approaches the subject of ear...
57,The Battleship Bismarck,The Bismarck is perhaps the most famous – and ...,['Stefan Draminski'],http://books.google.com/books/content?id=nxttD...,http://books.google.nl/books?id=nxttDwAAQBAJ&p...,Bloomsbury Publishing,20/09/2018,https://play.google.com/store/books/details?id...,['History'],1.0,0887402216,34.95,A30IUAABSEHEPI,Lawrence Duckles,9/9,3.0,1055980800,The Battleship Bismarck reviewed,This book is both a history and a photo album ...
58,The Battleship Bismarck,The Bismarck is perhaps the most famous – and ...,['Stefan Draminski'],http://books.google.com/books/content?id=nxttD...,http://books.google.nl/books?id=nxttDwAAQBAJ&p...,Bloomsbury Publishing,20/09/2018,https://play.google.com/store/books/details?id...,['History'],1.0,0887402216,34.95,A29SCEAL3JPMYW,Tim Grasshoff,0/0,4.0,1217808000,nice historical pictures,I like this book first one I've seen with so m...
74,Beginner's Yoruba (Hippocrene Beginner's Series),"""Beginner's Yoruba"" is now available with two ...",['Kayode J. Fakinlede'],http://books.google.com/books/content?id=xLe4n...,http://books.google.nl/books?id=xLe4nWzeSw0C&p...,Hippocrene Books,2005,http://books.google.nl/books?id=xLe4nWzeSw0C&d...,['Foreign Language Study'],1.0,0781810698,19.77,A1F0EV2MBF208I,"Olena Y. Rabinowitz ""Book lover""",14/14,4.0,1156291200,Very authentic,This is my first encounter with Yoruba and I h...


In [5]:
from sklearn.model_selection import train_test_split

X_polarite = data.drop(['Image', 'Lien Google', 'infoLink', 'Id', 'revue/score'], axis=1)
Y_polarite = data['revue/score']

X_train_polarite, X_test_polarite, Y_train_polarite, Y_test_polarite = train_test_split(X_polarite, Y_polarite, test_size=0.2, random_state=42)

In [6]:
from sklearn.preprocessing import OneHotEncoder, RobustScaler
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from scipy.sparse import hstack
import pandas as pd

# Identifier les types de colonnes
colonnes_categoriques = ['Auteurs', 'Editeur', 'Genre', 'Nom lecteur', 'revue/utilité']
colonnes_numeriques = ['Nb scores', 'Prix', 'revue/heure']
colonnes_textuelles = ['Titre', 'revue/résumé', 'revue/texte']

# Transformer les colonnes catégorielles et numériques
preprocessor = ColumnTransformer([
    ('cat', OneHotEncoder(handle_unknown='ignore'), colonnes_categoriques),
    ('num', RobustScaler(), colonnes_numeriques),
])

# Appliquer fit_transform sur X_train, mais SEULEMENT transform sur X_test
X_train_cat_num = preprocessor.fit_transform(X_train_polarite)
X_test_cat_num = preprocessor.transform(X_test_polarite)

# Appliquer TF-IDF sur chaque colonne textuelle séparément
text_pipelines = {col: TfidfVectorizer(max_features=500) for col in colonnes_textuelles}

X_train_text = [text_pipelines[col].fit_transform(X_train_polarite[col].fillna("").astype(str)) for col in colonnes_textuelles]
X_test_text = [text_pipelines[col].transform(X_test_polarite[col].fillna("").astype(str)) for col in colonnes_textuelles]

# Concaténer les matrices obtenues pour chaque colonne textuelle
X_train_text_combined = hstack(X_train_text)
X_test_text_combined = hstack(X_test_text)

# Vérification des dimensions après transformation
print(f"Shape X_train_cat_num: {X_train_cat_num.shape}")
print(f"Shape X_test_cat_num: {X_test_cat_num.shape}")
print(f"Shape X_train_text_combined: {X_train_text_combined.shape}")
print(f"Shape X_test_text_combined: {X_test_text_combined.shape}")

# Concaténer toutes les matrices ensemble
if X_train_cat_num.shape[0] == X_train_text_combined.shape[0] and X_test_cat_num.shape[0] == X_test_text_combined.shape[0]:
    X_train_encoded = hstack([X_train_cat_num, X_train_text_combined])
    X_test_encoded = hstack([X_test_cat_num, X_test_text_combined])

    # Vérification finale des dimensions
    print("✅ Shape X_train_encoded:", X_train_encoded.shape)
    print("✅ Shape X_test_encoded:", X_test_encoded.shape)
else:
    print("❌ Problème de dimensions entre les colonnes non-textuelles et textuelles.")


Shape X_train_cat_num: (12776, 21692)
Shape X_test_cat_num: (3195, 21692)
Shape X_train_text_combined: (12776, 1500)
Shape X_test_text_combined: (3195, 1500)
✅ Shape X_train_encoded: (12776, 23192)
✅ Shape X_test_encoded: (3195, 23192)


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

# Définir le modèle
model = Sequential()
model.add(Dense(128, activation='relu', input_shape=(X_train_encoded.shape[1],)))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='linear'))  # Utiliser 'linear' pour une régression

# Compiler le modèle
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

# Entraîner le modèle
history = model.fit(X_train_encoded.toarray(), Y_train_polarite, epochs=50, batch_size=32, validation_split=0.2)

# Évaluer le modèle
loss, mae = model.evaluate(X_test_encoded.toarray(), Y_test_polarite)
print(f"Test Loss: {loss}")
print(f"Test MAE: {mae}")

# Enregistrer le modèle
import pickle
filename = 'models/deepl.pkl'
pickle.dump(model, open(filename, 'wb'))

c:\Users\cedri\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/50
320/320 ━━━━━━━━━━━━━━━━━━━━ 13s 35ms/step - loss: 14.7939 - mae: 2.0324 - val_loss: 1.0379 - val_mae: 0.7873
Epoch 2/50
320/320 ━━━━━━━━━━━━━━━━━━━━ 11s 33ms/step - loss: 2.0005 - mae: 1.0432 - val_loss: 1.2813 - val_mae: 0.8140
Epoch 3/50
320/320 ━━━━━━━━━━━━━━━━━━━━ 10s 32ms/step - loss: 1.7796 - mae: 0.9619 - val_loss: 0.9920 - val_mae: 0.7859
Epoch 4/50
320/320 ━━━━━━━━━━━━━━━━━━━━ 10s 32ms/step - loss: 1.3114 - mae: 0.8760 - val_loss: 0.9540 - val_mae: 0.7536
Epoch 5/50
320/320 ━━━━━━━━━━━━━━━━━━━━ 10s 32ms/step - loss: 1.0325 - mae: 0.8026 - val_loss: 0.9663 - val_mae: 0.7891
Epoch 6/50
320/320 ━━━━━━━━━━━━━━━━━━━━ 10s 33ms/step - loss: 1.0070 - mae: 0.7793 - val_loss: 0.9630 - val_mae: 0.7859
Epoch 7/50
320/320 ━━━━━━━━━━━━━━━━━━━━ 11s 33ms/step - loss: 0.8676 - mae: 0.7383 - val_loss: 0.9316 - val_mae: 0.7506
Epoch 8/50
320/320 ━━━━━━━━━━━━━━━━━━━━ 11s 33ms/step - loss: 0.8544 - mae: 0.7179 - val_loss: 0.9292 - val_mae: 0.7549
Epoch 9/50
320/320 ━━━━━━━━━━━━━━━━━━━━

KeyboardInterrupt: 